<a href="https://colab.research.google.com/github/Khushi9898/Deployment-of-Glaucoma-Classification/blob/main/Deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# necessary for streamlit to work
!pip install -U ipykernel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.1 MB/s eta 0:00:00
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.17.1
    Uninstalling ipykernel-6.17.1:
      Successfully uninstalled ipykernel-6.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==6.17.1, but you have ipykernel 6.29.5 which is incompatible.


In [2]:
# allows to create web apps pretty easily
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.6 MB/s eta 0:00:00


In [3]:
# to expose local ip address as an external ip address
!pip install pyngrok

In [4]:
!ngrok config add-authtoken 2uFZtg7iqq1sJyNP9tzkjvTijDB_6W8z5HQeddmNSqxC69H8X

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [81]:
%%writefile score.py

import torch
import numpy as np
import streamlit as st  # Web app framework
from PIL import Image  # Image processing
import requests  # Fetch images from URLs
from io import BytesIO  # Convert image into bytes
import torchvision.transforms as transforms
import torchvision.models as models

st.title("Glaucoma Image Classifier")
st.text("Provide URL of a Fundus Image for Classification")

# Cache model to avoid reloading every time
@st.cache_resource
def load_model():
    model = models.googlenet(pretrained=False, aux_logits=True)  # Ensure aux_logits=True if trained with it
    model.fc = torch.nn.Linear(1024, 2)  # Match trained model's output layer

    # Load trained model weights
    state_dict = torch.load('/content/drive/MyDrive/PyTorch/Glaucoma_Detection/googlenet.pth',
                            map_location=torch.device('cpu'))

    model.load_state_dict(state_dict, strict=False)  # Allow minor mismatches
    model.eval()  # Set to evaluation mode
    return model

# Load the model once and reuse
with st.spinner('Loading Model Into Memory...'):
    model = load_model()

# Define class labels
classes = ['Normal', 'Glaucoma']

# Function to process image
def decode_image(image_bytes):
    image = Image.open(BytesIO(image_bytes)).convert("RGB")  # Decode JPEG and ensure RGB
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to model input size
        transforms.ToTensor(),  # Convert to tensor
    ])
    img = transform(image).unsqueeze(0)  # Add batch dimension (1, C, H, W)
    return img

# Get image URL from user input
path = st.text_input('Enter Image URL to Classify:',
                     "https://github.com/Khushi9898/Deployment-of-Glaucoma-Classification/blob/main/EyePACS-Glaucoma-3171%20(1).jpg?raw=true")

if path:
    try:
        content = requests.get(path).content  # Get byte array of the image

        # Display image
        image = Image.open(BytesIO(content))
        st.image(image, caption='Uploaded Image', use_container_width=True)

        # Model Prediction
        with st.spinner('Classifying...'):
            image_tensor = decode_image(content)  # Convert image to tensor
            with torch.no_grad():
                output = model(image_tensor)  # Forward pass
                label = torch.argmax(output, dim=1).item()  # Get predicted class

            st.write("**Predicted Class:**", classes[label])  # Display result

    except Exception as e:
        st.error(f"Error processing image: {e}")


Overwriting score.py


In [82]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [83]:
from pyngrok import ngrok

# List existing tunnels
for tunnel in ngrok.get_tunnels():
    print(tunnel)
    # Disconnect all existing tunnels
    ngrok.disconnect(tunnel.public_url)

NgrokTunnel: "https://a01d-34-125-5-136.ngrok-free.app" -> "http://localhost:8501"


In [84]:
# Define the tunnel configuration
tunnel_config = {
    "addr": 8501,  # Specify the port here
    "proto": "http"  # Or "tcp" if needed
}

# Connect with the configuration
url = ngrok.connect(**tunnel_config)
url

<NgrokTunnel: "https://3537-34-125-5-136.ngrok-free.app" -> "http://localhost:8501">

In [85]:
# show what happen when the program started
!cat nohup.out




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.5.136:8501

2025-03-13 07:30:48.296 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

/content/score.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrus